In [1]:
!pip install pycryptodome
!pip install matplotlib

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 558.9 kB/s eta 0:00:03
   ----------- ---------------------------- 0.5/1.8 MB 558.9 kB/s eta 0:00:03
   ----------------- ---------------------- 0.8/1.8 MB 568.6 kB/s eta 0:00:02
   ----------------- ---------------------- 0.8/1.8 MB 568.6 kB/s eta 0:00:0


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# crypto_tool.py
# CSE-478 Lab 4: Symmetric & Asymmetric Crypto
# Author: [Your Name]
# Credits:
# - PyCryptodome docs: https://www.pycryptodome.org/
# - RSA signing: https://pycryptodome.readthedocs.io/en/latest/src/signature/pkcs1_v1_5.html
# - AES modes: https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html

import os
import time
import hashlib
import matplotlib.pyplot as plt
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

KEYS_DIR = "keys"
OUTPUT_DIR = "output"

os.makedirs(KEYS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ====================== KEY MANAGEMENT ======================
def generate_aes_key(length=16):
    key = get_random_bytes(length)
    return key

def save_aes_key(key, filename):
    with open(os.path.join(KEYS_DIR, filename), 'wb') as f:
        f.write(key)

def load_aes_key(filename):
    path = os.path.join(KEYS_DIR, filename)
    if not os.path.exists(path):
        print(f"{filename} not found. Generating new key...")
        key = generate_aes_key(16 if '128' in filename else 32)
        save_aes_key(key, filename)
        return key
    with open(path, 'rb') as f:
        return f.read()

def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    
    with open(os.path.join(KEYS_DIR, "rsa_private.pem"), 'wb') as f:
        f.write(private_key)
    with open(os.path.join(KEYS_DIR, "rsa_public.pem"), 'wb') as f:
        f.write(public_key)
    print("RSA keys generated.")
    return key

def load_rsa_private():
    path = os.path.join(KEYS_DIR, "rsa_private.pem")
    if not os.path.exists(path):
        return generate_rsa_keys()
    with open(path, 'rb') as f:
        return RSA.import_key(f.read())

def load_rsa_public():
    path = os.path.join(KEYS_DIR, "rsa_public.pem")
    with open(path, 'rb') as f:
        return RSA.import_key(f.read())

# ====================== AES ======================
def aes_encrypt_file(key_length, mode, input_file, output_file):
    key = load_aes_key(f"aes_{key_length}_key.bin")
    iv = get_random_bytes(16)
    
    if mode == 'ECB':
        cipher = AES.new(key, AES.MODE_ECB)
    elif mode == 'CFB':
        cipher = AES.new(key, AES.MODE_CFB, iv=iv)
    
    start_time = time.time()
    
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    
    if mode == 'ECB':
        ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    else:  # CFB
        ciphertext = cipher.encrypt(plaintext)
    
    with open(output_file, 'wb') as f:
        f.write(iv if mode == 'CFB' else b'')  # Store IV only for CFB
        f.write(ciphertext)
    
    elapsed = time.time() - start_time
    print(f"AES-{key_length}-{mode} Encryption completed in {elapsed:.6f}s")
    return elapsed

def aes_decrypt_file(key_length, mode, input_file, output_file):
    key = load_aes_key(f"aes_{key_length}_key.bin")
    
    with open(input_file, 'rb') as f:
        if mode == 'CFB':
            iv = f.read(16)
            ciphertext = f.read()
            cipher = AES.new(key, AES.MODE_CFB, iv=iv)
        else:
            ciphertext = f.read()
            cipher = AES.new(key, AES.MODE_ECB)
    
    start_time = time.time()
    
    if mode == 'ECB':
        plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    else:
        plaintext = cipher.decrypt(ciphertext)
    
    with open(output_file, 'wb') as f:
        f.write(plaintext)
    
    elapsed = time.time() - start_time
    print(f"AES-{key_length}-{mode} Decryption completed in {elapsed:.6f}s")
    print(f"Decrypted content saved to {output_file}")
    print("Preview (first 200 chars):")
    print(plaintext[:200].decode('utf-8', errors='ignore'))
    return elapsed

# ====================== RSA ======================
def rsa_encrypt_file(input_file, output_file):
    public_key = load_rsa_public()
    cipher_rsa = PKCS1_OAEP.new(public_key)
    
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    
    start_time = time.time()
    ciphertext = cipher_rsa.encrypt(plaintext)
    elapsed = time.time() - start_time
    
    with open(output_file, 'wb') as f:
        f.write(ciphertext)
    
    print(f"RSA Encryption completed in {elapsed:.6f}s")
    return elapsed

def rsa_decrypt_file(input_file, output_file):
    private_key = load_rsa_private()
    cipher_rsa = PKCS1_OAEP.new(private_key)
    
    with open(input_file, 'rb') as f:
        ciphertext = f.read()
    
    start_time = time.time()
    plaintext = cipher_rsa.decrypt(ciphertext)
    elapsed = time.time() - start_time
    
    with open(output_file, 'wb') as f:
        f.write(plaintext)
    
    print(f"RSA Decryption completed in {elapsed:.6f}s")
    print("Decrypted content:")
    print(plaintext.decode('utf-8', errors='ignore'))
    return elapsed

# ====================== RSA SIGNATURE ======================
def rsa_sign_file(input_file, signature_file):
    private_key = load_rsa_private()
    with open(input_file, 'rb') as f:
        data = f.read()
    
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)
    
    with open(signature_file, 'wb') as f:
        f.write(signature)
    
    print(f"Signature generated and saved to {signature_file}")
    return True

def rsa_verify_file(input_file, signature_file):
    public_key = load_rsa_public()
    with open(input_file, 'rb') as f:
        data = f.read()
    with open(signature_file, 'rb') as f:
        signature = f.read()
    
    h = SHA256.new(data)
    try:
        pkcs1_15.new(public_key).verify(h, signature)
        print("Signature is VALID.")
        return True
    except (ValueError, TypeError):
        print("Signature is INVALID.")
        return False

# ====================== SHA-256 ======================
def sha256_file(input_file):
    h = hashlib.sha256()
    with open(input_file, 'rb') as f:
        while chunk := f.read(8192):
            h.update(chunk)
    hash_hex = h.hexdigest()
    print(f"SHA-256: {hash_hex}")
    with open(os.path.join(OUTPUT_DIR, "hash.txt"), "w") as f:
        f.write(hash_hex)
    return hash_hex

# ====================== PERFORMANCE TEST ======================
def performance_test():
    print("\n" + "="*50)
    print("PERFORMANCE TEST: Execution Time vs Key Size")
    print("="*50)
    
    # Create test file
    test_file = "test_input.txt"
    with open(test_file, "w") as f:
        f.write("A" * 10000)  # 10KB data
    
    aes_times = {}
    rsa_times = {}
    sizes = [128, 256, 512, 1024, 2048]  # bits
    
    # AES Performance
    for n in [128, 256]:
        key = get_random_bytes(n//8)
        save_aes_key(key, f"temp_aes_{n}.bin")
        times = []
        for _ in range(3):
            t = aes_encrypt_file(n, 'ECB', test_file, f"temp_enc_{n}.bin")
            times.append(t)
        aes_times[n] = sum(times)/len(times)
    
    # RSA Performance (key size = N bits)
    rsa_key_sizes = sizes
    for n in rsa_key_sizes:
        key = RSA.generate(n)
        pub = key.publickey().export_key()
        priv = key.export_key()
        with open("keys/temp_pub.pem", "wb") as f: f.write(pub)
        with open("keys/temp_priv.pem", "wb") as f: f.write(priv)
        
        with open(test_file, "rb") as f:
            data = f.read()[:n//8 - 11]  # Fit into RSA block
        
        with open("temp_small.txt", "wb") as f:
            f.write(data)
        
        times = []
        for _ in range(3):
            start = time.time()
            cipher = PKCS1_OAEP.new(RSA.import_key(pub))
            cipher.encrypt(data)
            times.append(time.time() - start)
        rsa_times[n] = sum(times)/len(times)
    
    # Plot
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    aes_bits = list(aes_times.keys())
    aes_t = list(aes_times.values())
    plt.plot(aes_bits, aes_t, 'bo-')
    plt.title('AES Encryption Time vs Key Size')
    plt.xlabel('Key Size (bits)')
    plt.ylabel('Time (seconds)')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    rsa_bits = list(rsa_times.keys())
    rsa_t = list(rsa_times.values())
    plt.plot(rsa_bits, rsa_t, 'ro-')
    plt.title('RSA Encryption Time vs Key Size')
    plt.xlabel('Key Size (bits)')
    plt.ylabel('Time (seconds)')
    plt.yscale('log')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig("performance_graph.png")
    plt.show()
    
    print("Graph saved as performance_graph.png")
    
    # Cleanup
    os.remove(test_file)
    for f in os.listdir("."):
        if f.startswith("temp_"):
            try: os.remove(f)
            except: pass
    for f in os.listdir(KEYS_DIR):
        if f.startswith("temp_"):
            os.remove(os.path.join(KEYS_DIR, f))

def main():
    print("\n=== CSE-478 Lab 4: Crypto Tool ===")
    print("1. AES Encrypt (128/256, ECB/CFB)")
    print("2. AES Decrypt")
    print("3. RSA Encrypt")
    print("4. RSA Decrypt")
    print("5. RSA Sign File")
    print("6. RSA Verify Signature")
    print("7. SHA-256 Hash File")
    print("8. Performance Test & Plot")
    print("9. Exit")
    
    while True:
        choice = input("\nChoose option (1-9): ").strip()
        
        if choice == '1':
            kl = input("Key length (128 or 256): ")
            mode = input("Mode (ECB or CFB): ").upper()
            infile = input("Input file: ")
            outfile = os.path.join(OUTPUT_DIR, "encrypted_aes.bin")
            if kl in ['128', '256'] and mode in ['ECB', 'CFB']:
                aes_encrypt_file(kl, mode, infile, outfile)
            else:
                print("Invalid key length or mode!")
        
        elif choice == '2':
            kl = input("Key length (128 or 256): ")
            mode = input("Mode (ECB or CFB): ").upper()
            infile = os.path.join(OUTPUT_DIR, "encrypted_aes.bin")
            outfile = os.path.join(OUTPUT_DIR, "decrypted.txt")
            if os.path.exists(infile):
                aes_decrypt_file(kl, mode, infile, outfile)
            else:
                print("Encrypted file not found!")
        
        elif choice == '3':
            infile = input("Input file: ")
            outfile = os.path.join(OUTPUT_DIR, "encrypted_rsa.bin")
            rsa_encrypt_file(infile, outfile)
        
        elif choice == '4':
            infile = os.path.join(OUTPUT_DIR, "encrypted_rsa.bin")
            outfile = os.path.join(OUTPUT_DIR, "decrypted_rsa.txt")
            if os.path.exists(infile):
                rsa_decrypt_file(infile, outfile)
            else:
                print("No RSA encrypted file found!")
        
        elif choice == '5':
            infile = input("File to sign: ")
            sigfile = os.path.join(OUTPUT_DIR, "signature.bin")
            rsa_sign_file(infile, sigfile)
        
        elif choice == '6':
            infile = input("Original file: ")
            sigfile = os.path.join(OUTPUT_DIR, "signature.bin")
            if os.path.exists(sigfile):
                rsa_verify_file(infile, sigfile)
            else:
                print("Signature file not found!")
        
        elif choice == '7':
            infile = input("File to hash: ")
            if os.path.exists(infile):
                sha256_file(infile)
            else:
                print("File not found!")
        
        elif choice == '8':
            performance_test()
        
        elif choice == '9':
            print("Goodbye!")
            break
        
        else:
            print("Invalid option!")

if __name__ == "__main__":
    # Generate keys on first run
    load_aes_key("aes_128_key.bin")
    load_aes_key("aes_256_key.bin")
    load_rsa_private()  # generates if not exists
    main()


=== CSE-478 Lab 4: Crypto Tool ===
1. AES Encrypt (128/256, ECB/CFB)
2. AES Decrypt
3. RSA Encrypt
4. RSA Decrypt
5. RSA Sign File
6. RSA Verify Signature
7. SHA-256 Hash File
8. Performance Test & Plot
9. Exit
AES-128-ECB Encryption completed in 0.010786s
Invalid key length or mode!
Invalid option!
Invalid option!
Invalid option!
Invalid option!
